# Scratch Notebook

## Create pandas/spark dataframes for testing

In [1]:
import numpy as np
import pandas as pd

from pyspark.sql.functions import col, pandas_udf, spark_partition_id, struct
from pyspark.sql.types import *
from typing import Iterator, Union

In [2]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)
pdf = pd.DataFrame(data, columns=['a','b','c','d'])

In [3]:
pdf_scalar = pdf
df_scalar = spark.createDataFrame(pdf_scalar)

pdf_tensor = pd.DataFrame()
pdf_tensor['t1'] = pdf_scalar.values.tolist()
df_tensor1 = spark.createDataFrame(pdf_tensor)

pdf_tensor['t2'] = pdf_scalar.drop(columns='d').values.tolist()
df_tensor2 = spark.createDataFrame(pdf_tensor)

### pandas dataframes

In [4]:
pdf_scalar

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
4,16.0,17.0,18.0,19.0
...,...,...,...,...
245,980.0,981.0,982.0,983.0
246,984.0,985.0,986.0,987.0
247,988.0,989.0,990.0,991.0
248,992.0,993.0,994.0,995.0


In [5]:
pdf_tensor

,t1,t2
0,"[0.0, 1.0, 2.0, 3.0]","[0.0, 1.0, 2.0]"
1,"[4.0, 5.0, 6.0, 7.0]","[4.0, 5.0, 6.0]"
2,"[8.0, 9.0, 10.0, 11.0]","[8.0, 9.0, 10.0]"
3,"[12.0, 13.0, 14.0, 15.0]","[12.0, 13.0, 14.0]"
4,"[16.0, 17.0, 18.0, 19.0]","[16.0, 17.0, 18.0]"
...,...,...
245,"[980.0, 981.0, 982.0, 983.0]","[980.0, 981.0, 982.0]"
246,"[984.0, 985.0, 986.0, 987.0]","[984.0, 985.0, 986.0]"
247,"[988.0, 989.0, 990.0, 991.0]","[988.0, 989.0, 990.0]"
248,"[992.0, 993.0, 994.0, 995.0]","[992.0, 993.0, 994.0]"


In [6]:
foo = pdf_scalar['a']

In [7]:
type(foo)

pandas.core.series.Series

In [9]:
bar = pdf_scalar['b']

In [10]:
type(bar)

pandas.core.series.Series

In [15]:
def batch(n=1):
    for i in range(0, len(foo), n):
        yield foo.iloc[i:i+n], bar.iloc[i:i+n]

In [17]:
for x in batch(2):
    print(x)

(0    0.0
1    4.0
Name: a, dtype: float64, 0    1.0
1    5.0
Name: b, dtype: float64)
(2     8.0
3    12.0
Name: a, dtype: float64, 2     9.0
3    13.0
Name: b, dtype: float64)
(4    16.0
5    20.0
Name: a, dtype: float64, 4    17.0
5    21.0
Name: b, dtype: float64)
(6    24.0
7    28.0
Name: a, dtype: float64, 6    25.0
7    29.0
Name: b, dtype: float64)
(8    32.0
9    36.0
Name: a, dtype: float64, 8    33.0
9    37.0
Name: b, dtype: float64)
(10    40.0
11    44.0
Name: a, dtype: float64, 10    41.0
11    45.0
Name: b, dtype: float64)
(12    48.0
13    52.0
Name: a, dtype: float64, 12    49.0
13    53.0
Name: b, dtype: float64)
(14    56.0
15    60.0
Name: a, dtype: float64, 14    57.0
15    61.0
Name: b, dtype: float64)
(16    64.0
17    68.0
Name: a, dtype: float64, 16    65.0
17    69.0
Name: b, dtype: float64)
(18    72.0
19    76.0
Name: a, dtype: float64, 18    73.0
19    77.0
Name: b, dtype: float64)
(20    80.0
21    84.0
Name: a, dtype: float64, 20    81.0
21    85.0
Name

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/leey/devpub/leewyang/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/leey/devpub/leewyang/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/leey/devpub/leewyang/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [9]:
baz = pd.concat((foo,), axis=1)

In [20]:
baz.iloc[:,0]

0        0.0
1        4.0
2        8.0
3       12.0
4       16.0
       ...  
245    980.0
246    984.0
247    988.0
248    992.0
249    996.0
Name: a, Length: 250, dtype: float64

In [21]:
baz.to_numpy()

array([[  0.],
       [  4.],
       [  8.],
       [ 12.],
       [ 16.],
       [ 20.],
       [ 24.],
       [ 28.],
       [ 32.],
       [ 36.],
       [ 40.],
       [ 44.],
       [ 48.],
       [ 52.],
       [ 56.],
       [ 60.],
       [ 64.],
       [ 68.],
       [ 72.],
       [ 76.],
       [ 80.],
       [ 84.],
       [ 88.],
       [ 92.],
       [ 96.],
       [100.],
       [104.],
       [108.],
       [112.],
       [116.],
       [120.],
       [124.],
       [128.],
       [132.],
       [136.],
       [140.],
       [144.],
       [148.],
       [152.],
       [156.],
       [160.],
       [164.],
       [168.],
       [172.],
       [176.],
       [180.],
       [184.],
       [188.],
       [192.],
       [196.],
       [200.],
       [204.],
       [208.],
       [212.],
       [216.],
       [220.],
       [224.],
       [228.],
       [232.],
       [236.],
       [240.],
       [244.],
       [248.],
       [252.],
       [256.],
       [260.],
       [26

### spark dataframes

In [ ]:
df_scalar.show()

In [ ]:
df_tensor1.show()

In [ ]:
df_tensor2.show()

### spark partitions

In [ ]:
from pyspark.sql.functions import spark_partition_id
df.withColumn("partition_id", spark_partition_id()).groupBy("partition_id").count().show()

## Test tensor columns

### pDF -> pDF | returnType=StructType | return pDF => OK

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=df_tensor2.schema)
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
columns = df_tensor2.columns
preds = df_tensor2.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pDF | returnType=ArrayType(DoubleType()) | return pS => OK

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=ArrayType(DoubleType()))
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch['t1']

In [ ]:
columns = df_tensor2.columns
preds = df_tensor2.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pS | returnType=StructType | return pDF => OK

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=df_tensor2.schema)
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.Series]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
columns = df_tensor2.columns
preds = df_tensor2.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pS | returnType=ArrayType(DoubleType()) | return pS => OK

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=ArrayType(DoubleType()))
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch['t1']

In [ ]:
columns = df_tensor2.columns
preds = df_tensor2.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### Union[pDF, pS] -> Union[pDF, pS] | returnType=StructType() | return pS => FAIL

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=df_tensor2.schema)
# def predict(inputs: Iterator[Union[pd.DataFrame, pd.Series]]) -> Iterator[Union[pd.DataFrame, pd.Series]]:
# def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[Union[pd.DataFrame, pd.Series]]:
# def predict(inputs: Iterator[pd.Series]) -> Iterator[Union[pd.DataFrame, pd.Series]]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

## Test scalar columns

### pDF -> pDF | returnType=StructType | return pDF => OK

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=df_scalar.schema)
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pDF | returnType=DoubleType() | return pS => OK

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=DoubleType())
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch['a']

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pS | returnType=StructType | return pDF => OK

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=df_scalar.schema)
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.Series]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pS | returnType=DoubleType() | return pS => OK

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=DoubleType())
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.Series]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch['a']

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### Union[pDF, pS] -> Union[pDF, pS] | returnType=StructType) | return pS => FAIL

In [ ]:
@pandas_udf(returnType=df_scalar.schema)
def predict(inputs: Iterator[Union[pd.DataFrame, pd.Series]]) -> Iterator[Union[pd.DataFrame, pd.Series]]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
from inspect import signature
from pyspark.sql.pandas.typehints import infer_eval_type
from typing import get_type_hints, Any, Callable, Optional, Tuple

In [ ]:
def check_iterator_annotation(
    annotation: Any, parameter_check_func: Optional[Callable[[Any], bool]] = None
) -> bool:
    name = getattr(annotation, "_name", getattr(annotation, "__name__", None))
    return name == "Iterator" and (
        parameter_check_func is None or all(map(parameter_check_func, annotation.__args__))
    )

In [ ]:
def check_union_annotation(
    annotation: Any, parameter_check_func: Optional[Callable[[Any], bool]] = None
) -> bool:
    import typing

    # Note that we cannot rely on '__origin__' in other type hints as it has changed from version
    # to version. For example, it's abc.Iterator in Python 3.7 but typing.Iterator in Python 3.6.
    origin = getattr(annotation, "__origin__", None)
    return origin == typing.Union and (
        parameter_check_func is None or all(map(parameter_check_func, annotation.__args__))
    )

In [ ]:
def check_tuple_annotation(
    annotation: Any, parameter_check_func: Optional[Callable[[Any], bool]] = None
) -> bool:
    # Tuple has _name but other types have __name__
    # Check if the name is Tuple first. After that, check the generic types.
    name = getattr(annotation, "_name", getattr(annotation, "__name__", None))
    return name == "Tuple" and (
        parameter_check_func is None or all(map(parameter_check_func, annotation.__args__))
    )

In [ ]:
def predict(inputs: Iterator[Union[pd.Series, pd.DataFrame]]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
sig = signature(predict)
sig

In [ ]:
type_hints = get_type_hints(predict)
type_hints

In [ ]:
annotations = {}
for param in sig.parameters.values():
    if param.annotation is not param.empty:
        annotations[param.name] = type_hints.get(param.name, param.annotation)
annotations

In [ ]:
parameters_sig = [
    annotations[parameter] for parameter in sig.parameters if parameter in annotations]
parameters_sig

In [ ]:
return_annotation = type_hints.get("return", sig.return_annotation)
return_annotation

In [ ]:
is_series_or_frame = all(
    a == pd.Series
    or a == pd.DataFrame  # Series
    or check_union_annotation(  # DataFrame  # Union[DataFrame, Series]
        a, parameter_check_func=lambda na: na == pd.Series or na == pd.DataFrame
    )
    for a in parameters_sig
) and (return_annotation == pd.Series or return_annotation == pd.DataFrame)
is_series_or_frame

In [ ]:
is_iterator_tuple_series_or_frame = (
    len(parameters_sig) == 1
    and check_iterator_annotation(  # Iterator
        parameters_sig[0],
        parameter_check_func=lambda a: check_tuple_annotation(  # Tuple
            a,
            parameter_check_func=lambda ta: (
                ta == Ellipsis
                or ta == pd.Series  # ...
                or ta == pd.DataFrame  # Series
                or check_union_annotation(  # DataFrame  # Union[DataFrame, Series]
                    ta, parameter_check_func=lambda na: (na == pd.Series or na == pd.DataFrame)
                )
            ),
        ),
    )
    and check_iterator_annotation(
        return_annotation, parameter_check_func=lambda a: a == pd.DataFrame or a == pd.Series
    )
)
is_iterator_tuple_series_or_frame

In [ ]:
is_iterator_series_or_frame = (
    len(parameters_sig) == 1
    and check_iterator_annotation(
        parameters_sig[0],
        parameter_check_func=lambda a: (
            a == pd.Series
            or a == pd.DataFrame  # Series
            or check_union_annotation(  # DataFrame  # Union[DataFrame, Series]
                a, parameter_check_func=lambda ua: ua == pd.Series or ua == pd.DataFrame
            )
        ),
    )
    and check_iterator_annotation(
        return_annotation, parameter_check_func=lambda a: a == pd.DataFrame or a == pd.Series
    )
)
is_iterator_series_or_frame

### pS -> pS | returnType=ArrayType(DoubleType()) | return pS => OK

In [ ]:
from pyspark.sql.functions import array

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=ArrayType(DoubleType()))
def predict(inputs: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        # print("===== batch.columns: {}".format(batch.columns))
        # print("===== batch.dtypes:\n{}".format(batch.dtypes))
        print("===== batch[0]:\n{}".format(batch[0]))
        yield batch

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(array(columns))).toPandas()
preds

## Test caching

In [ ]:
import numpy as np
import pandas as pd
import threading
import time

from pyspark.ml.functions import batch_infer_udf
from pyspark.sql.functions import struct, pandas_udf
from pyspark.sql.types import *
from typing import Iterator

In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)

# 4 scalar columns
pdf = pd.DataFrame(data, columns=["a", "b", "c", "d"])
df = spark.createDataFrame(pdf)

In [ ]:
def predict_batch_fn():
    # emulate loading a model, this should only be invoked once (per worker process)
    fake_output = np.random.random()

    def predict(inputs):
        return [fake_output for i in inputs]

    return predict

In [ ]:
identity = batch_infer_udf(predict_batch_fn, return_type=DoubleType())

In [ ]:
%%time
# results should be the same
df1 = df.withColumn("preds", identity(struct("a"))).toPandas()

In [ ]:
%%time
df2 = df.withColumn("preds", identity(struct("a"))).toPandas()

In [ ]:
%%time
df2 = df.withColumn("preds", identity(struct("a"))).toPandas()

In [ ]:
%%time
df2 = df.withColumn("preds", identity(struct("a"))).toPandas()

In [ ]:
%%time
df2 = df.withColumn("preds", identity(struct("a"))).toPandas()

## Test executors

In [ ]:
import numpy as np
import pandas as pd
import threading
import time

from pyspark.ml.functions import batch_infer_udf
from pyspark.sql.functions import struct, pandas_udf
from pyspark.sql.types import *
from typing import Iterator

In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)

# 4 scalar columns
pdf = pd.DataFrame(data, columns=["a", "b", "c", "d"])
df = spark.createDataFrame(pdf)

In [ ]:
def myfn(it):
    import tensorflow as tf
    print(">>>> {}".format(tf.__version__))
    print(tf.config.list_physical_devices('GPU'))
    tf.debugging.set_log_device_placement(True)

    # Create some tensors
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
    print(c)

In [ ]:
df.foreachPartition(myfn)

In [ ]:
@pandas_udf(returnType=FloatType())
def myudf(it: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for part in it:
        import tensorflow as tf
        print(tf.__version__)
        yield part

In [ ]:
foo = df.withColumn("preds", struct("a"))

In [ ]:
foo.show()

In [ ]:
print("==== df1:\n{}".format(df1))
print("==== df2:\n{}".format(df2))

In [ ]:
from pyspark.sql.functions import spark_partition_id
df.withColumn("partition_id", spark_partition_id()).withColumn("preds", identity(struct("a"))).groupBy("partition_id", "preds").count().show()

## Test zip

In [4]:
import pandas as pd

In [36]:
foo = pd.Series([0,1,2,3,4,5,6,7,8])
bar = pd.Series(['a','b','c','d','e','f','g','h','i'])

In [37]:
test = (foo, bar)
test

(0    0
 1    1
 2    2
 3    3
 4    4
 5    5
 6    6
 7    7
 8    8
 dtype: int64,
 0    a
 1    b
 2    c
 3    d
 4    e
 5    f
 6    g
 7    h
 8    i
 dtype: object)

In [31]:
def batch(iterable, batch_size):
    acc = []
    for i, x in enumerate(zip(*iterable)):
        acc += x
        if i % batch_size == 0:
            yield acc
            acc = []
    yield acc

In [38]:
import itertools
import more_itertools

def batch(iterable, batch_size):
    for x in more_itertools.chunked(zip(*iterable), batch_size):
        yield x

In [39]:
for x in batch(test, 2):
    print(x)
    print("====")

[(0, 'a'), (1, 'b')]
====
[(2, 'c'), (3, 'd')]
====
[(4, 'e'), (5, 'f')]
====
[(6, 'g'), (7, 'h')]
====
[(8, 'i')]
====


In [50]:
pdf = pd.concat(test, axis=1)

In [51]:
pdf

,0,1
0,0,a
1,1,b
2,2,c
3,3,d
4,4,e
5,5,f
6,6,g
7,7,h
8,8,i


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/leey/devpub/leewyang/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/leey/devpub/leewyang/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/leey/devpub/leewyang/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
/home/leey/devpub/leewyang/spark/python/pyspark/context.py:642: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been kille